### Part 4 - Load the chosen model & Extract its embeddings

Set the version data control parameter (to save the outputs of this notebook at their latest date)

In [ ]:
version_data_control="14082020"

#### Import the libraries

In [ ]:
import collections

try:
    collectionsAbc = collections.abc
except AttributeError:
    collectionsAbc = collections

import pandas as pd
import numpy as np
import os
import time
import random
import pickle
import json
import shutil
import unidecode
import string
import itertools

%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from datetime import datetime
from tabulate import tabulate
from packaging import version
from humanfriendly import format_timespan
from sklearn.metrics import confusion_matrix, classification_report, hamming_loss, zero_one_loss, f1_score, roc_auc_score

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
date_format='%Y-%m-%d %H-%M-%S'

#### Import Tensorflow

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_docs as tfdocs #!pip install git+https://github.com/tensorflow/docs
import tensorflow_docs.plots as tfplots
import tensorflow_docs.modeling as tfmodel

from tensorflow.keras import layers, regularizers, models
from tensorflow.keras import models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import model_to_dot, plot_model
from tensorflow.keras.models import load_model, model_from_json

#### Import Tensorflow

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_docs as tfdocs #!pip install git+https://github.com/tensorflow/docs
import tensorflow_docs.plots as tfplots
import tensorflow_docs.modeling as tfmodel

from tensorflow.keras import layers, regularizers, models
from tensorflow.keras import models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import model_to_dot, plot_model
from tensorflow.keras.models import load_model, model_from_json

#### Import Tensorflow Hub

In [ ]:
import tensorflow_hub as hub
from tensorboard.plugins.hparams import api as hp

#### Import Keras

In [ ]:
import keras.backend as K
from tensorflow import keras

In [ ]:
print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

## <b>- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -  </b>

#### Import the dataset from part 3.1

In [ ]:
dataset = pd.read_pickle(os.path.join(os.getcwd(), "pickled_data_per_part\\dataset_part_3.1_13072020.pkl"))
dataset.shape

In [ ]:
dataset['clean_actors'].iloc[0].split(",")

#### Load the already trained chosen model
This is the model that perfomed better than those trained on part 3. <br>
<i> The chosen model is the "Multi-input keras model". <i>

In [ ]:
folder_path_model_saved="model_one\\adam_v2_models_20072020"
saved_model_name="multi_input_keras_model"
saved_model_date="16072020"

HP_HIDDEN_UNITS = hp.HParam('batch_size', hp.Discrete([32]))
HP_EMBEDDING_DIM = hp.HParam('embedding_dim', hp.Discrete([100]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.001]))
HP_DECAY_STEPS_MULTIPLIER = hp.HParam('decay_steps_multiplier', hp.Discrete([10]))

In [ ]:
"""
Load the weights of the best model selected from previous part 3.3
"""
def import_weights_keras_model(method, optimizer, hparams):
    """
    Load the weights of the model saved with EarlyStopping
    """
    if method == "import custom trained model":
        
        if optimizer == "adam":
        
            with open(os.path.join(os.getcwd(), "{0}\\{1}_{2}dim_{3}batchsize_{4}lr_{5}decaymultiplier_{6}.json".format(folder_path_model_saved,
                                                                                                                        saved_model_name,
                                                                                                                        str(hparams[HP_EMBEDDING_DIM]), 
                                                                                                                        str(hparams[HP_HIDDEN_UNITS]), 
                                                                                                                        str(hparams[HP_LEARNING_RATE]), 
                                                                                                                        str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                                        saved_model_date)),'r') as f:
                model_json = json.load(f)

            model_imported = model_from_json(model_json)

            model_imported.load_weights(os.path.join(os.getcwd(), '{0}\\{1}_{2}dim_{3}batchsize_{4}lr_{5}decaymultiplier_{6}.h5'.format(folder_path_model_saved,
                                                                                                                                        saved_model_name,
                                                                                                                                        str(hparams[HP_EMBEDDING_DIM]), 
                                                                                                                                        str(hparams[HP_HIDDEN_UNITS]), 
                                                                                                                                        str(hparams[HP_LEARNING_RATE]), 
                                                                                                                                        str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                                                        saved_model_date)))
            print(type(model_imported))
            print("\nModel's weights have been loaded successfully\n")
            
        else:
        
            with open(os.path.join(os.getcwd(), "{0}\\{1}_{2}dim_{3}batchsize_{4}lr_{5}.json".format(folder_path_model_saved,
                                                                                                     saved_model_name,
                                                                                                     str(hparams[HP_EMBEDDING_DIM]), 
                                                                                                     str(hparams[HP_HIDDEN_UNITS]), 
                                                                                                     str(hparams[HP_LEARNING_RATE]),
                                                                                                     saved_model_date)),'r') as f:
                model_json = json.load(f)

            model_imported = model_from_json(model_json)

            model_imported.load_weights(os.path.join(os.getcwd(), '{0}\\{1}_{2}dim_{3}batchsize_{4}lr_{5}.h5'.format(folder_path_model_saved,
                                                                                                                     saved_model_name,
                                                                                                                     str(hparams[HP_EMBEDDING_DIM]),
                                                                                                                     str(hparams[HP_HIDDEN_UNITS]),
                                                                                                                     str(hparams[HP_LEARNING_RATE]),
                                                                                                                     saved_model_date)))
            print(type(model_imported))
            print("\nModel's weights have been loaded successfully\n")
    
    else:
        
        with open(os.path.join(os.getcwd(), "{0}\\{1}_{2}dim_{3}batchsize_{4}lr_{5}decaymultiplier_{6}.json".format(folder_path_model_saved,
                                                                                                                    saved_model_name,
                                                                                                                    str(hparams[HP_EMBEDDING_DIM]), 
                                                                                                                    str(hparams[HP_HIDDEN_UNITS]), 
                                                                                                                    str(hparams[HP_LEARNING_RATE]), 
                                                                                                                    str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                                    saved_model_date)),'r') as f:
            model_json = json.load(f)

        model_imported = model_from_json(model_json, custom_objects={'KerasLayer':hub.KerasLayer})

        model_imported.load_weights(os.path.join(os.getcwd(), '{0}\\{1}_{2}dim_{3}batchsize_{4}lr_{5}decaymultiplier_{6}.h5'.format(folder_path_model_saved,
                                                                                                                                    saved_model_name,
                                                                                                                                    str(hparams[HP_EMBEDDING_DIM]), 
                                                                                                                                    str(hparams[HP_HIDDEN_UNITS]), 
                                                                                                                                    str(hparams[HP_LEARNING_RATE]), 
                                                                                                                                    str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                                                    saved_model_date)))
        print(type(model_imported))
        print("\nModel is loaded successfully\n")
    
    return model_imported

In [ ]:
for batch_size in HP_HIDDEN_UNITS.domain.values:
    for embedding_dim in HP_EMBEDDING_DIM.domain.values:
        for learning_rate in HP_LEARNING_RATE.domain.values:
            for decay_steps_multiplier in HP_DECAY_STEPS_MULTIPLIER.domain.values:
                hparams = {
                    HP_HIDDEN_UNITS: batch_size,
                    HP_EMBEDDING_DIM: embedding_dim,
                    HP_LEARNING_RATE: learning_rate,
                    HP_DECAY_STEPS_MULTIPLIER: decay_steps_multiplier
                  }
                model = import_weights_keras_model("import custom trained model", "adam", hparams)

#### Import the saved tokenizers

In [ ]:
saved_version_data_control="13072020"
tokenization_history_folder="text_tokenization_padded_sequences"

with open(os.path.join(os.getcwd(), '80-20 split_non-balanced\\{0}_{1}\\words_tokenized_{1}.pkl'.format(tokenization_history_folder, saved_version_data_control)), 'rb') as handle:
    words_tokenized = pickle.load(handle)

In [ ]:
"""
IMport the tokenizers of each input, fitted on part 3.1
"""
with open(os.path.join(os.getcwd(), '80-20 split_non-balanced\\{0}_{1}\\actors_tokenizer_{2}_{1}.pkl'.format(tokenization_history_folder, saved_version_data_control, words_tokenized['actors_tokenized'])),'rb') as f:
    actors_tokenizer = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '80-20 split_non-balanced\\{0}_{1}\\plot_tokenizer_{2}_{1}.pkl'.format(tokenization_history_folder, saved_version_data_control, words_tokenized['plot_words_tokenized'])),'rb') as f:
    plot_tokenizer = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '80-20 split_non-balanced\\{0}_{1}\\features_tokenizer_{2}_{1}.pkl'.format(tokenization_history_folder, saved_version_data_control, words_tokenized['features_words_tokenized'])),'rb') as f:
    features_tokenizer = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '80-20 split_non-balanced\\{0}_{1}\\reviews_tokenizer_{2}_{1}.pkl'.format(tokenization_history_folder, saved_version_data_control, words_tokenized['reviews_words_tokenized'])),'rb') as f:
    reviews_tokenizer = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '80-20 split_non-balanced\\{0}_{1}\\title_tokenizer_{2}_{1}.pkl'.format(tokenization_history_folder, saved_version_data_control, words_tokenized['title_words_tokenized'])),'rb') as f:
    title_tokenizer = pickle.load(f)

try:
    assert len(actors_tokenizer.word_index)==words_tokenized['actors_tokenized']
    assert len(plot_tokenizer.word_index)==words_tokenized['plot_words_tokenized']
    assert len(features_tokenizer.word_index)==words_tokenized['features_words_tokenized']
    assert len(reviews_tokenizer.word_index)==words_tokenized['reviews_words_tokenized']
    assert len(title_tokenizer.word_index)==words_tokenized['title_words_tokenized']
except AssertionError:
    print("ERROR: The vocabulary length for some of the tokenizers, is not equal to 20000. Please verify their lengths by running the following: len(actors_tokenizer.word_index)")

print("Tokenizers are loaded successfully!")

#### Two Fucntions have been assembled to complete the word embeddings extraction

In [ ]:
def extract_word_embeddings(variable, model, tokenizer):
    
    if variable == "actors":
        
        embeddings_layer = model.layers[5].get_weights()[0]

        word_embeddings = {w:embeddings_layer[idx] for w, idx in tokenizer.word_index.items() if idx <= len(tokenizer.word_index)+1}
        
        print("\nActor's word embeddings length: {}".format(embeddings_layer.shape))
        
    elif variable == "plot":
        
        embeddings_layer = model.layers[6].get_weights()[0]

        word_embeddings = {w:embeddings_layer[idx] for w, idx in tokenizer.word_index.items() if idx <= len(tokenizer.word_index)+1}
        print("Plot Summary's word embeddings length: {}".format(embeddings_layer.shape))
        
    elif variable == "features":
        
        embeddings_layer = model.layers[7].get_weights()[0]

        word_embeddings = {w:embeddings_layer[idx] for w, idx in tokenizer.word_index.items() if idx <= len(tokenizer.word_index)+1}
        print("Movie's Features word embeddings length: {}".format(embeddings_layer.shape))
        
    elif variable == "reviews":
        
        embeddings_layer = model.layers[8].get_weights()[0]

        word_embeddings = {w:embeddings_layer[idx] for w, idx in tokenizer.word_index.items() if idx <= len(tokenizer.word_index)+1}
        print("Movie's Reviews word embeddings length: {}".format(embeddings_layer.shape))
        
    elif variable == "title":
        
        embeddings_layer = model.layers[9].get_weights()[0]

        word_embeddings = {w:embeddings_layer[idx] for w, idx in tokenizer.word_index.items() if idx <= len(tokenizer.word_index)+1}
        print("Movie's title word embeddings length: {}".format(embeddings_layer.shape))
        
    return embeddings_layer, word_embeddings

def assign_word_embeddings(variable, dataset, word_embeddings):
    
    if variable == "actors":
    
        average_vector_list_cast = []

        min_vector_list_cast = []

        max_vector_list_cast = []

        actors_embeddings_list = []

        for i in range(len(dataset.index)):
            
            actors = dataset.loc[:, "clean_actors"].iloc[i].split(",")
    
            assert [word.islower() for word in actors] # assert that all actors are present in lower case
    
            actors_embeddings_list.append([word_embeddings[word] if word in word_embeddings else word_embeddings['<OOV>'] for word in actors])
    
        dataset.loc[:, 'actors_embeddings_list'] = actors_embeddings_list

        for i in range(len(dataset.index)):
            
            actor_embeddings = dataset["actors_embeddings_list"].iloc[i]
    
            minimum = np.min([element for element in actor_embeddings], axis=0)
            maximum = np.max([element for element in actor_embeddings], axis=0)
            average = np.mean([element for element in actor_embeddings], axis=0)
    
            min_vector_list_cast.append(minimum)
            max_vector_list_cast.append(maximum)
            average_vector_list_cast.append(average)

        dataset['minimum_cast_vectors'] = min_vector_list_cast
        dataset['maximum_cast_vectors'] = max_vector_list_cast
        dataset['average_cast_vectors'] = average_vector_list_cast
        
    elif variable == "plot":
    
        average_vector_list_plot = []

        min_vector_list_plot = []

        max_vector_list_plot = []

        plot_embeddings_list = []

        for i in range(len(dataset.index)):
            
            plot = dataset["clean_plot_summary"].iloc[i]
    
            plot_split = plot.split(' ')
    
            assert [word.islower() for word in plot_split]
    
            plot_embeddings_list.append([word_embeddings[word] if word in word_embeddings else word_embeddings['<OOV>'] for word in plot_split])
    
        dataset['plot_embeddings_list'] = plot_embeddings_list

        for i in range(len(dataset.index)):
            
            plot_embeddings = dataset["plot_embeddings_list"].iloc[i]
    
            minimum = np.min([element for element in plot_embeddings], axis=0)
            maximum = np.max([element for element in plot_embeddings], axis=0)
            average = np.mean([element for element in plot_embeddings], axis=0)
    
            min_vector_list_plot.append(minimum)
            max_vector_list_plot.append(maximum)
            average_vector_list_plot.append(average)

        dataset['minimum_plot_vectors'] = min_vector_list_plot
        dataset['maximum_plot_vectors'] = max_vector_list_plot
        dataset['average_plot_vectors'] = average_vector_list_plot
        
    elif variable == "features":
    
        average_vector_list_combined_features = []

        min_vector_list_combined_features = []

        max_vector_list_combined_features = []

        combined_features_embeddings_list = []

        for i in range(len(dataset.index)):
    
            combined_features = dataset["clean_combined_features"].iloc[i]
    
            combined_features_split = combined_features.split(' ')
    
            assert [word.islower() for word in combined_features_split]
    
            combined_features_embeddings_list.append([word_embeddings[word] if word in word_embeddings else word_embeddings['<OOV>'] for word in combined_features_split])
    
        dataset['combined_features_embeddings_list'] = combined_features_embeddings_list

        for i in range(len(dataset.index)):
            
            combined_features_embeddings = dataset["combined_features_embeddings_list"].iloc[i]
    
            minimum = np.min([element for element in combined_features_embeddings], axis=0)
            maximum = np.max([element for element in combined_features_embeddings], axis=0)
            average = np.mean([element for element in combined_features_embeddings], axis=0)
    
            min_vector_list_combined_features.append(minimum)
            max_vector_list_combined_features.append(maximum)
            average_vector_list_combined_features.append(average)

        dataset['minimum_combined_features_vectors'] = min_vector_list_combined_features
        dataset['maximum_combined_features_vectors'] = max_vector_list_combined_features
        dataset['average_combined_features_vectors'] = average_vector_list_combined_features

    elif variable == "reviews":
    
        average_vector_list_reviews = []

        min_vector_list_reviews = []

        max_vector_list_reviews = []

        reviews_embeddings_list = []

        for i in range(len(dataset.index)):
    
            reviews = dataset["clean_reviews"].iloc[i]
    
            reviews_split = reviews.split(' ')
    
            assert [word.islower() for word in reviews_split]

            reviews_embeddings_list.append([word_embeddings[word] if word in word_embeddings else word_embeddings['<OOV>'] for word in reviews_split])

        dataset['reviews_embeddings_list'] = reviews_embeddings_list

        for i in range(len(dataset.index)):
            
            reviews_embeddings = dataset["reviews_embeddings_list"].iloc[i]
    
            minimum = np.min([element for element in reviews_embeddings], axis=0)
            maximum = np.max([element for element in reviews_embeddings], axis=0)
            average = np.mean([element for element in reviews_embeddings], axis=0)
    
            min_vector_list_reviews.append(minimum)
            max_vector_list_reviews.append(maximum)
            average_vector_list_reviews.append(average)

        dataset['minimum_reviews_vectors'] = min_vector_list_reviews
        dataset['maximum_reviews_vectors'] = max_vector_list_reviews
        dataset['average_reviews_vectors'] = average_vector_list_reviews
        
    elif variable == "title":
    
        average_vector_list_title = []

        min_vector_list_title = []

        max_vector_list_title = []

        title_embeddings_list = []

        for i in range(len(dataset.index)):
    
            title = dataset["clean_movie_title"].iloc[i]
    
            title_split = title.split(' ')
    
            assert [word.islower() for word in title_split]

            title_embeddings_list.append([word_embeddings[word] if word in word_embeddings else word_embeddings['<OOV>'] for word in title_split])

        dataset['title_embeddings_list'] = title_embeddings_list

        for i in range(len(dataset.index)):
            
            title_embeddings = dataset["title_embeddings_list"].iloc[i]
    
            minimum = np.min([element for element in title_embeddings], axis=0)
            maximum = np.max([element for element in title_embeddings], axis=0)
            average = np.mean([element for element in title_embeddings], axis=0)
    
            min_vector_list_title.append(minimum)
            max_vector_list_title.append(maximum)
            average_vector_list_title.append(average)

        dataset['minimum_title_vectors'] = min_vector_list_title
        dataset['maximum_title_vectors'] = max_vector_list_title
        dataset['average_title_vectors'] = average_vector_list_title

In [ ]:
# Extract the word embeddings
print("\n---------------------------------------------------------------------------------")
print("\nExtract the word embeddings")
begin_time=time.time()

actors_embedding_layer, word_embeddings_actors = extract_word_embeddings("actors", model, actors_tokenizer)
print("Word embeddings for actors extracted\n")

plot_embedding_layer, word_embeddings_plot = extract_word_embeddings("plot", model, plot_tokenizer)
print("Word embeddings for plot summary extracted\n")

features_embedding_layer, word_embeddings_features = extract_word_embeddings("features", model, features_tokenizer)
print("Word embeddings for movie features extracted\n")

reviews_embedding_layer, word_embeddings_reviews = extract_word_embeddings("reviews", model, reviews_tokenizer)
print("Word embeddings for movie reviews extracted\n")

title_embedding_layer, word_embeddings_title = extract_word_embeddings("title", model, title_tokenizer)
print("Word embeddings for movie reviews extracted\n")

print("Finished word embeddings extraction in: {0}".format(format_timespan(time.time() - begin_time)))

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

In [ ]:
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# Assign the word embeddings to each different actor
print("\n---------------------------------------------------------------------------------")
print("\nAssign the word embeddings to each different word\n")
begin_time=time.time()

print("Assign word embeddings to actors")
assign_word_embeddings("actors", dataset, word_embeddings_actors)

print("Assign word embeddings to plot summary")
assign_word_embeddings("plot", dataset, word_embeddings_plot)

print("Assign word embeddings to movie features")
assign_word_embeddings("features", dataset, word_embeddings_features)

print("Assign word embeddings to movie reviews")
assign_word_embeddings("reviews", dataset, word_embeddings_reviews)

print("Assign word embeddings to movie titles")
assign_word_embeddings("title", dataset, word_embeddings_title)

print("\nFinished word embeddings extraction in: {0}".format(format_timespan(time.time() - begin_time)))

In [ ]:
print("\nThe word embedding vector of the actor 'tobey maguire' is:\n\n", word_embeddings_actors['tobey maguire'])

In [ ]:
begin_time=time.time()
keras_embeddings_array_cast = np.hstack([dataset['average_cast_vectors'].apply(pd.Series).values,
                                         dataset['minimum_cast_vectors'].apply(pd.Series).values,
                                         dataset['maximum_cast_vectors'].apply(pd.Series).values])

keras_embeddings_array_plot = np.hstack([dataset['average_plot_vectors'].apply(pd.Series).values,
                                         dataset['minimum_plot_vectors'].apply(pd.Series).values,
                                         dataset['maximum_plot_vectors'].apply(pd.Series).values])

keras_embeddings_array_combined_features = np.hstack([dataset['average_combined_features_vectors'].apply(pd.Series).values,
                                                      dataset['minimum_combined_features_vectors'].apply(pd.Series).values,
                                                      dataset['maximum_combined_features_vectors'].apply(pd.Series).values])

keras_embeddings_array_reviews = np.hstack([dataset['average_reviews_vectors'].apply(pd.Series).values,
                                            dataset['minimum_reviews_vectors'].apply(pd.Series).values,
                                            dataset['maximum_reviews_vectors'].apply(pd.Series).values])

keras_embeddings_array_title = np.hstack([dataset['average_title_vectors'].apply(pd.Series).values,
                                          dataset['minimum_title_vectors'].apply(pd.Series).values,
                                          dataset['maximum_title_vectors'].apply(pd.Series).values])

keras_embeddings_array_cast_plot_combined_features_reviews_title = np.hstack([keras_embeddings_array_cast, 
                                                                              keras_embeddings_array_plot, 
                                                                              keras_embeddings_array_combined_features,
                                                                              keras_embeddings_array_reviews,
                                                                              keras_embeddings_array_title])

print("Shape of the Actors embeddings: {}".format(keras_embeddings_array_cast.shape))
print("\nShape of the Plot Summary embeddings: {}".format(keras_embeddings_array_plot.shape))
print("\nShape of the Combined Features embeddings: {}".format(keras_embeddings_array_combined_features.shape))
print("\nShape of the Reviews embeddings: {}".format(keras_embeddings_array_reviews.shape))
print("\nShape of the Title embeddings: {}".format(keras_embeddings_array_title.shape))

print("\nShape of the concatenated embeddings(cast, plot, combined features, reviews, title): {}".format(keras_embeddings_array_cast_plot_combined_features_reviews_title.shape))

print("\nFinished word embeddings extraction in: {0}".format(format_timespan(time.time() - begin_time)))

In [ ]:
"""
Since the chosen model is the "Model_1: Multi-Input Keras Model", we saved the relevant word embeddings to the folder "model_one"
"""
with open(os.path.join(os.getcwd(), 'final_word_embeddings_conversational_agent\\keras_embeddings_array_cast_{0}.pkl'.format(version_data_control)), 'wb') as f:
    pickle.dump(keras_embeddings_array_cast, f)
    
with open(os.path.join(os.getcwd(), 'final_word_embeddings_conversational_agent\\keras_embeddings_array_plot_{0}.pkl'.format(version_data_control)), 'wb') as f:
    pickle.dump(keras_embeddings_array_plot, f)
    
with open(os.path.join(os.getcwd(), 'final_word_embeddings_conversational_agent\\keras_embeddings_array_combined_features_{0}.pkl'.format(version_data_control)), 'wb') as f:
    pickle.dump(keras_embeddings_array_combined_features, f)
    
with open(os.path.join(os.getcwd(), 'final_word_embeddings_conversational_agent\\keras_embeddings_array_reviews_{0}.pkl'.format(version_data_control)), 'wb') as f:
    pickle.dump(keras_embeddings_array_reviews, f)
    
with open(os.path.join(os.getcwd(), 'final_word_embeddings_conversational_agent\\keras_embeddings_array_title_{0}.pkl'.format(version_data_control)), 'wb') as f:
    pickle.dump(keras_embeddings_array_title, f)
    
with open(os.path.join(os.getcwd(), 'final_word_embeddings_conversational_agent\\keras_embeddings_array_concatenated_{0}.pkl'.format(version_data_control)), 'wb') as f:
    pickle.dump(keras_embeddings_array_cast_plot_combined_features_reviews_title, f)

In [ ]:
keras_embeddings_array_cast_plot_combined_features_reviews_title[0].shape

#### Pickle the dataset with the word embeddings

In [ ]:
dataset.columns

In [ ]:
dataset_part_5=dataset[["title", "actors", "clean_combined_features", "clean_reviews", "imdb_rating", "sentiment_value", "rating", "imdb_url", "reduced_genres"]]

In [ ]:
dataset_part_5.iloc[0:5]

In [ ]:
dataset_part_5.to_pickle(os.path.join(os.getcwd(), 'pickled_data_per_part\\dataset_part_4_{0}.pkl'.format(version_data_control)))

### END OF Part 4 - Load the chosen model & Extract Word Embeddings